In [13]:
#import copy
import sys
import numpy as np
import open3d as o3d
sys.path.append(".")
#import pyransac3d as pyrsc
import flattten_pcd as fp
import matplotlib.pyplot as plt
load_pcd = o3d.io.read_point_cloud("point_pipe.pcd")
print("length pcd ", len(np.array(load_pcd.points)))
o3d.visualization.draw_geometries([load_pcd])

length pcd  1826


In [14]:
print("Downsample the point cloud with a voxel of 0.02")
pcd2 = load_pcd.voxel_down_sample(voxel_size=0.02)
print("length downpcd ", len(np.array(pcd2.points)))
o3d.visualization.draw_geometries([pcd2])

Downsample the point cloud with a voxel of 0.02
length downpcd  433


In [16]:
plane_model, inliers = pcd2.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd2.select_by_index(inliers)
print(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd2.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 1.0, 0])
print(len(inliers), len(np.array(pcd2.points)))

Plane equation: 0.97x + 0.00y + -0.26z + -0.62 = 0
[6, 7, 8, 9, 10, 11, 20, 22, 23, 24, 25, 26, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 91, 92, 93, 94, 95, 96, 97, 102, 103, 105, 106, 107, 108, 109, 110, 112, 113, 115, 117, 120, 122, 124, 126, 128, 130, 132, 133, 134, 139, 140, 143, 147, 148, 149, 150, 151, 152, 155, 157, 158, 159, 161, 164, 170, 171, 173, 176, 178, 179, 180, 182, 183, 186, 188, 190, 191, 192, 193, 195, 196, 197, 198, 199, 202, 204, 207, 211, 213, 215, 218, 219, 220, 221, 226, 229, 230, 231, 233, 237, 240, 242, 246, 248, 250, 253, 256, 258, 259, 261, 265, 266, 267, 272, 276, 279, 282, 285, 286, 289, 293, 296, 297, 299, 302, 303, 305, 311, 312, 313, 314, 315, 319, 322, 323, 324, 325, 326, 328, 330, 331, 332, 333, 334, 338, 341, 343, 344, 346, 353, 355, 357, 358, 359, 360, 361, 362, 366, 367, 368, 370, 372, 374, 377, 378, 379, 380, 381, 385, 389, 391, 392, 

In [18]:
print("inlider")
o3d.visualization.draw_geometries([inlier_cloud])
print("outlier_cloud")
o3d.visualization.draw_geometries([outlier_cloud])

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

inlider
outlier_cloud
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
Precompute neighbors.[========================================] 100%


In [19]:
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
print("length label:  ",len(labels))
print("length outlier_cloud:  ",len(np.array(outlier_cloud.points)))
o3d.visualization.draw_geometries([outlier_cloud])

point cloud has 0 clusters
length label:   209
length outlier_cloud:   209


In [9]:
cluster = o3d.geometry.PointCloud()
for c in range(max_label+1):
    cluster = [i for i, n in enumerate(labels.tolist()) if n == c]
    print(cluster) 
    cluster = outlier_cloud.select_by_index(cluster)
    print("length cluster:  ",len(np.array(cluster.points)))
    o3d.visualization.draw_geometries([cluster])

In [8]:
cluster = [i for i, n in enumerate(labels.tolist()) if n == 1]
print(cluster) 
cluster = outlier_cloud.select_by_index(cluster)
print("length cluster:  ",len(np.array(cluster.points)))
o3d.visualization.draw_geometries([cluster])

[]
length cluster:   0


In [10]:
import cylinder2 as cy2

ModuleNotFoundError: No module named 'aux_functions'

In [11]:
cil = cy2.Cylinder()
points = np.asarray(cluster.points)
center, normal, radius, inliers = cil.fit(points, thresh=0.01, maxIteration=10000)

#center, normal, radius, inliers = fp.get_cylinder(cluster, thresh=0.01, maxIteration=10000)
mesh_cylinder = fp.get_clylinder_mesh(center, normal, radius)
print("center: " + str(center))
print("radius: " + str(radius))
print("vecC: " + str(normal))
#print("inliers: ", inliers)

inlier_cloud = cluster.select_by_index(inliers)

NameError: name 'cy2' is not defined

In [16]:
o3d.visualization.draw_geometries([mesh_cylinder,inlier_cloud])

NameError: name 'mesh_cylinder' is not defined